In [1]:
!pip install emoji
!pip install -q pandarallel
!sudo apt-get -o Dpkg::Options::="--force-confmiss" install --reinstall netbase
!pip install git+https://github.com/crazyfrogspb/RedditScore.git
!pip install extraction

In [2]:
import pandas as pd
import sys
import json
import os
import extraction
import requests
from google.colab import drive
from redditscore.tokenizer import CrazyTokenizer
import nltk
import re
import emoji
import requests
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import warnings
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('punkt') # if necessary...
pd.set_option('display.max_colwidth', 255)
warnings.filterwarnings("ignore")
nltk.download('stopwords')
drive.mount('/content/drive')

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/COVID19 Fake News Detection in English/input/'
test_df = pd.read_excel(f"{path}Constraint_English_Test.xlsx")
test_df

,id,tweet
0,1,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He Were To Catch The Coronavirus https://t.co/3MEWhusRZI #donaldtrump #coronavirus
3,4,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process. https://t....
4,5,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros https://t.co/JvKC0PTett
...,...,...
2135,2136,#CoronaVirusUpdates: State-wise details of Total Confirmed #COVID19 cases (till 16 September 2020 8 AM) ➡️States with 1-15000 confirmed cases ➡️States with 15001-115000 confirmed cases ➡️States with 115000+ confirmed cases ➡️Total no. of confirmed cas...
2136,2137,"Tonight 12(midnight) onwards Disaster Management Act has been implemented across the country. According to this update, apart from the Govt department no other citizen is allowed to post any update or share any forward related to Coronavirus, it being..."
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 Enugu-46 Oyo-31 Lagos-21 Rivers-20 FCT-15 Kaduna-13 Bauchi-12 Delta-11 Ekiti-11 Akwa Ibom-7 Ebonyi-6 Kwara-5 Ogun-4 Osun-4 Gombe-3 Niger-2 53317 confirmed 40726 discharged 1011 deaths https://t.co/7VLNB37mw1
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Shop for #COVID19 Resources has a section for communicating with people ages 15-21. Find those t…


In [5]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]
print(cosine_sim('politically correct woman (almost) uses pandemic as excuse not to reuse plastic bag  #coronavirus #nashville', 'politically correct woman almost uses pandemic as excuse not to reuse plastic bag | the spoof'))
#print(cosine_sim('a little bird', 'None'))
# print(cosine_sim('a little bird', 'a big dog barks'))

0.7548969126692828


In [7]:
def _store_data(load_data,link):
  store_path = f"{path}{link}.json"
  if os.path.exists(store_path) == False:
      store_data = []
      store_data.append(load_data)
      with open(store_path, 'w') as fp:
          json.dump(store_data, fp)
  else:
      with open(store_path, 'r') as fp:
          data = json.load(fp)
      data.append(load_data)
      with open(store_path, 'w') as fp:
          json.dump(data, fp)
  return

def text_replace(df,link):
  len_df = len(df)
  for index in range(len_df):
    text = df['tweet'][index]
    urls = re.findall('(https?://[^\s]+)',  df['tweet'][index])
    data = dict()
    data['index'] = index
    for url in urls:
      try:
        url = re.findall('.*[^.]', url)[0]
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        data[url] = extracted.title
      except:
        continue
    if index % 500 == 0:
      print(index)
    _store_data(data,link)
text_replace(test_df,"test_link_store")

2000


In [8]:
def politifact_count(link):
  path = f"/content/drive/My Drive/COVID19 Fake News Detection in English/input/{link}.json"
  with open(path, 'r') as fp:
    data = json.load(fp)
  politifact = 0
  poli_list = []
  for obj in data:
    keys = list(obj.keys())
    keys.remove('index')
    if len(keys) > 0:
      for url in keys:
        if obj[url] != None:
          text = obj[url].lower()
          text = text.split(' ')
          if 'politifact' in text:
            poli_list.append(obj['index'])
            politifact += 1
  return poli_list,len(data), politifact

test_poli_list,test_len, test_poli = politifact_count("test_link_store")
print(f"politifact website exist in validation data is = {test_poli/test_len * 100}num is {test_poli}/{test_len}")

politifact website exist in validation data is = 1.6822429906542056num is 36/2140


In [9]:
def without_link_tweet(text):
  raw_text = re.sub(r'(https?://[^\s]+)', '', text, flags=re.MULTILINE)
  return raw_text
test_df['raw_tweet'] = test_df.parallel_apply(lambda x: without_link_tweet(x['tweet']), axis=1)

   0.00%                                          |        0 /     1070 |      
   0.00%                                          |        0 /     1070 |      M   0.93%                                          |       10 /     1070 |      
   0.00%                                          |        0 /     1070 |      M   4.02% :                                        |       43 /     1070 |      
   0.93%                                          |       10 /     1070 |      M   4.02% :                                        |       43 /     1070 |      
  97.94% :::::::::::::::::::::::::::::::::::::::  |     1048 /     1070 |      M 100.00% :::::::::::::::::::::::::::::::

In [10]:
test_df.sample(5)

,id,tweet,raw_tweet
495,496,RT @PIB_India: #CoronaWatch ◾ 3387500 total confirmed cases ◾ 2583948 cases cured/recovered ◾ 39477848 samples tested Here's the…,RT @PIB_India: #CoronaWatch ◾ 3387500 total confirmed cases ◾ 2583948 cases cured/recovered ◾ 39477848 samples tested Here's the…
1146,1147,These people have been assessed as presenting a very low risk due to the nature of their exemption adherence to their required protocols and the negative test results of people associated with their bubble.,These people have been assessed as presenting a very low risk due to the nature of their exemption adherence to their required protocols and the negative test results of people associated with their bubble.
946,947,Our daily update is published. We’ve now tracked 16.9 million tests up 441k from yesterday the 2nd-highest total we've recorded. Note that we can only track tests that a state reports. For details see: https://t.co/PZrmH4bl5Y https://t.co/BT1YANufXT,Our daily update is published. We’ve now tracked 16.9 million tests up 441k from yesterday the 2nd-highest total we've recorded. Note that we can only track tests that a state reports. For details see:
527,528,As of 30 July 2020 there are 477 active #COVID19 cases in #Begusarai District #Bihar. For district-specific details kindly contact District #COVID Control Room: 06243-222835 https://t.co/l9arr3Y7NA,As of 30 July 2020 there are 477 active #COVID19 cases in #Begusarai District #Bihar. For district-specific details kindly contact District #COVID Control Room: 06243-222835
250,251,@FamiliaFeeling There is a daily number you can get at any time. Then there is the current number. We tend to do 3 runs per day. The big midday run (where we go A—&gt;Z) then update-only runs at night and in the morning.,@FamiliaFeeling There is a daily number you can get at any time. Then there is the current number. We tend to do 3 runs per day. The big midday run (where we go A—&gt;Z) then update-only runs at night and in the morning.


In [11]:
test_df['clean_tweet'] = test_df['tweet']

In [12]:
def link_replace(df, link):
  path = f"/content/drive/My Drive/COVID19 Fake News Detection in English/input/{link}.json"
  with open(path, 'r') as fp:
    data = json.load(fp)
  len_df = len(df)
  for index in range(len_df):
    raw_text = re.sub(r'(https?://[^\s]+)', '', df['clean_tweet'][index], flags=re.MULTILINE)
    urls = re.findall('(https?://[^\s]+)',df['clean_tweet'][index])
    if len(urls) > 0:
      for obj in data:
        if index == obj['index']:
          for url in urls:
            try:
              location_text = obj[url]
              if location_text == None:
                df['clean_tweet'][index] = re.sub(url, '', df['clean_tweet'][index])
              elif cosine_sim(raw_text.lower(), location_text.lower()) > 0.7:
                df['clean_tweet'][index] = re.sub(url, '', df['clean_tweet'][index])
              else:
                df['clean_tweet'][index] = re.sub(url, location_text, df['clean_tweet'][index])
            except:
              pass
  return df
test_df = link_replace(test_df,"test_link_store")

In [13]:
test_df.sample(5)

,id,tweet,raw_tweet,clean_tweet
965,966,We also have to report a bug in our rolling average lines in some previous charts. In some cases they were calculating over a longer time period which made changes harder to see. We regret the error.,We also have to report a bug in our rolling average lines in some previous charts. In some cases they were calculating over a longer time period which made changes harder to see. We regret the error.,We also have to report a bug in our rolling average lines in some previous charts. In some cases they were calculating over a longer time period which made changes harder to see. We regret the error.
895,896,Video of dead bodies being piled up in Italy due to COVID-19.,Video of dead bodies being piled up in Italy due to COVID-19.,Video of dead bodies being piled up in Italy due to COVID-19.
876,877,Here are the states reporting over 400 cases today. https://t.co/lJdzm1EuSR,Here are the states reporting over 400 cases today.,Here are the states reporting over 400 cases today.
266,267,“Shands Hospital of Gainesville Florida has confirmed its first case of coronavirus.”,“Shands Hospital of Gainesville Florida has confirmed its first case of coronavirus.”,“Shands Hospital of Gainesville Florida has confirmed its first case of coronavirus.”
159,160,Claim: The ‘Fund of Funds’ part of the #COVID19 stimulus will mobilise equity of ~Rs 50000 cr for MSMEs Fact: A similar fund set up in 2016 to mobilise Rs 60000 cr for start-ups disbursed only 6% of planned amount https://t.co/wF9lG2PM0F,Claim: The ‘Fund of Funds’ part of the #COVID19 stimulus will mobilise equity of ~Rs 50000 cr for MSMEs Fact: A similar fund set up in 2016 to mobilise Rs 60000 cr for start-ups disbursed only 6% of planned amount,"Claim: The ‘Fund of Funds’ part of the #COVID19 stimulus will mobilise equity of ~Rs 50000 cr for MSMEs Fact: A similar fund set up in 2016 to mobilise Rs 60000 cr for start-ups disbursed only 6% of planned amount Before MSME ‘Fund Of Funds’, A Rs60,0..."


In [14]:
final_hash = []
def hash_tag_count(df):
  hash_tag_words = []
  hash_tag = df.clean_tweet.str.findall(r'#.*?(?=\s|$)')
  for val in hash_tag:
    if (len(val) != 0):
      words = [word for word in val]
      hash_tag_words.extend(words)
  return hash_tag_words, Counter(hash_tag_words), len(hash_tag_words)
test_hash,hash_tag_words_test, count_test = hash_tag_count(test_df)
print(hash_tag_words_test)
print(count_test)
final_hash.extend(test_hash)

Counter({'#COVID19': 393, '#coronavirus': 131, '#IndiaFightsCorona': 62, '#IndiaFightsCorona:': 52, '#StaySafe': 52, '#IndiaWillWin': 47, '#COVID19.': 44, '#CoronaVirusUpdates:': 36, '#COVID': 29, '#TakeResponsibility': 26, '#CoronaVirusUpdates': 25, '#covid19': 25, '#COVIDー19': 24, '#COVID__19': 20, '#donaldtrump': 17, '#CoronavirusIndia': 17, '#Covid_19': 15, '#COVID19Nigeria;': 15, '#lockdown': 14, '#CoronaUpdates': 13, '#COVIDUpdates': 13, '#COVID_19': 12, '#CoronavirusPandemic': 11, '#CoronavirusFacts': 11, '#COVID19Nigeria': 10, '#COVID19India': 10, '#COVID19;': 9, '#WearAMask': 8, '#COVIDView': 8, '#Corona': 7, '#Coronavirus': 7, '#china': 7, '#COVID19:': 7, '#pandemic': 6, '#KayBurley': 6, '#COVID19?': 6, '#RecoveryRate': 5, '#CoronaWatch': 5, '#coronavirusfacts': 5, '#Covid19': 5, '#HCPs:': 5, '#coronaoutbreak': 4, '#KayBurley:': 4, '#datoscoronavirus': 4, '#COVIDStopsWithMe': 4, '#DYK?': 4, '#covid_19': 3, '#facemasks': 3, '#CoronavirusUpdates': 3, '#coronavirusupdate': 3, '#

In [15]:
emojis = []
for txt in test_df['tweet']:
  text = emoji.demojize(txt)
  text = re.findall(r'(:[!_\-\w]+:)', text)
  list_emoji = [emoji.emojize(x) for x in text]
  emojis.extend(list_emoji)

In [16]:
print(emojis)

['🇰🇼', '👇', '👉', '🤝', '🖊️', '⚫', '😷', '😷', '😷', '😷', '😷', '😷', '✅', '✅', '✅', '🌍', '📩', '⬆️', '📑', '👇', '🐄', '💦', '💁🏿\u200d♂️', '📢', '📍', '➡️', '➡️', '👍', '➡️', '➡️', '▶️', '▶️', '▶️', '👉', '📢', '🇵🇬', '👉', '👉', '▪️', '▪️', '▪️', '▪️', '▪️', '📍', '📢', '✅', '📍', '👐', '\U0001f9fc', '\U0001f9fc', '🌐', '👉', '👉', '➡️', '➡️', '👇', '👇', '◾', '◾', '◾', '📍', '▶️', '▶️', '▶️', '📢', '✅', '📍', '😕', '😥', '📢', '✅', '📍', '📢', '📍', '➡️', '➡️', '👍', '➡️', '➡️', '👉', '😐', '😐', '😐', '👉', '👉', '👇🏽', '📢', '📍', '➡️', '➡️', '👍', '➡️', '➡️', '👉', '👉', '👉🏼', '🔲', '🔲', '◾', '◾', '◾', '😏', '📍', '▶️', '▶️', '▶️', '📍', '👇', '🔒', '🏠', '◾', '◾', '◾', '◾', '◾', '◾', '🔴', '🔴', '🔴', '📍', '▶️', '▶️', '▶️', '✅', '✅', '✅', '😳', '⚠', '👉', '👉', '1️⃣', '2️⃣', '🆕', '🌏', '🌍', '🌎', '👉', '😀', '✔️', '✔️', '✔️', '➡️', '🇿🇦', '💡', '2⃣', '8⃣', '2⃣', '0⃣', '👉', '🔴', '🔴', '🔴', '👉', '📺', '📱', '➡️', '➡️', '➡️', '🔰', '📢', '✅', '📍', '📍', '📌', '✅', '😢', '🙏🏻', '📍', '▶️', '▶️', '▶️', '👇', '✅', '✅', '✅', '👇', '👉', '▶️', '▶️', '▶️', '▶️', '➡️', 

In [22]:
def replace_emoji(text):
    text = text.lower()
    text = re.sub(r"👑🦠",  " coronavirus ", text)
    text = re.sub(r"📍", " update ", text)
    text = re.sub(r"🆕", " update ", text)
    text = re.sub(r"📌", " update ", text) 
    #text = re.sub(r"✅", " verified", text)
    text = re.sub(r"📢", " update ", text)
    text = re.sub(r"📣", " update ", text)
    text = re.sub(r"🔴", " warning ", text)
    text = re.sub(r"👍", " approved ", text)
    text = re.sub(r"🔰", " driving sign ", text)
    text = re.sub(r"🙏", " thank you ", text)
    text = re.sub(r"😷", " wear a mask ", text)
    text = re.sub(r"❗", " excitement ", text)
    text = re.sub(r"🌍", " europe africa ", text)
    text = re.sub(r"🌎", " america ", text)
    text = re.sub(r"🌏", " asia austrila ", text) 
    text = re.sub(r"🌐", " globe ", text) 
    text = re.sub(r"🏫", " school ", text)
    text = re.sub(r"📺", " television ", text)
    text = re.sub(r"👏", " hand washing ", text)
    text = re.sub(r"🤔", " thinking ", text)
    text = re.sub(r"🏠", " stay at home ", text)
    text = re.sub(r"🏡", " stay at home ", text)
    text = re.sub(r"🕌", " mosque ", text)
    text = re.sub(r"📈", " increasing ", text)
    text = re.sub(r"📉", " decreasing ", text) 
    text = re.sub(r"🙌", " hand washing ", text)
    text = re.sub(r"👐", " hand washing ", text)
    text = re.sub(r"👦", " boy ", text)
    text = re.sub(r"❌", " not ", text)
    text = re.sub(r"🙄", " rolling eye ", text)
    text = re.sub(r"📏", " keep distance ", text)
    text = re.sub(r"👥", " social distance ", text)
    text = re.sub(r"💉", " vaccination ", text)
    text = re.sub(r"\U0001f9ea", "vaccination ", text)
    text = re.sub(r"🧪", "vaccination ", text) 
    text = re.sub(r"💊", " pill ", text)
    text = re.sub(r"🗣️", " speak ", text) 
    text = re.sub(r"🚍", " transportation ", text)
    text = re.sub(r"🚌", " transportation ", text)
    text = re.sub(r"💺", " seat ", text) 
    text = re.sub(r"🐍", " snake ", text)
    text = re.sub(r"💊", " mike ", text)
    text = re.sub(r"🏥", " hospital ", text) 
    text = re.sub(r"🚑", " ambulance ", text)  
    text = re.sub(r"🚨", " emergency ", text)
    text = re.sub(r"🏪", " convenience store ", text)
    text = re.sub(r"😞", " disappointed ", text) 
    text = re.sub(r"🎉", " party popper ",text )
    text = re.sub(r"📰", " newspaper ",text )
    text = re.sub(r"🤱", " breast feeding ",text)
    text = re.sub(r"👀", " astonished ",text)
    text = re.sub(r"🧤", " gloves ",text)
    text = re.sub(r"🚶🏿", " social distance ",text)
    text = re.sub(r"🚥", " traffic light ",text)
    text = re.sub(r"↗", " up right ",text)
    text = re.sub(r"⚕", " medicine and health care services ",text)
    
    # text = re.sub(r"🙂", " smile",text)
    # text = re.sub(r"😁", " smile",text)
    # text = re.sub(r"😠", " angry",text)
    # text = re.sub(r"😭", " crying",text)
    # text = re.sub(r"😧", " worry",text) #anguished
    # text = re.sub(r"😢", " crying",text)
    # text = re.sub(r"😣", " helplessness",text)#persevering_face
    # text = re.sub(r"😞", " disappointed",text)
    # text = re.sub(r"😖", " sad",text)
    # text = re.sub(r"😬", " disapproval",text)
    # text = re.sub(r"🤣", " laughing",text)
    # text = re.sub(r"😐", " neutral",text)
    
    text = re.sub(r"↔️", " keep distance ", text)
    text = re.sub(r"🦠", " virus ", text)
    text = re.sub(r"💧", " droplet ", text)
    text = re.sub(r"💦", " droplet ", text)
    text = re.sub(r"🧴", " sanitizer ", text)
    text = re.sub(r"🧼", " hand wash ", text)
    text = re.sub(r"🚫", " not ", text)
    text = re.sub(r"🚰", " hand wash ", text)
    text = re.sub(r"🤝", " not handshake ", text)
    text = re.sub(r"🧑‍🤝‍🧑", " not close contact ", text)
    text = re.sub(r"🧫", " test for covid-19 ", text)
    text = re.sub(r"🧪", " test for covid-19 ", text)
    text = re.sub(r"🌡️", " body temperature measured ", text)
    text = re.sub(r"🔬", " test for covid-19 ", text)
    text = re.sub(r"📝", " contact tracing ", text)
    text = re.sub(r"🧑‍⚕️", " health worker ", text)
    text = re.sub(r"🐦", " hand wash ", text)
    text = re.sub(r"🛡", " shielding required ", text)
    text = re.sub(r"♿️", " accessibility requirements ", text)
    text = re.sub(r"🆘", " emergency help required ", text)    

    with open('/content/drive/My Drive/COVID19 Fake News Detection in English/country_emoji.json') as json_file:
      data = json.load(json_file)
    for val in data:
      name = f" {val['name']} "
      text = re.sub(val['emoji'], name, text)
    text = text.strip(' ')
    return text
def digit_remove(text):
  text = re.sub("\d+", " ", text)
  #text = text.replace('$$', "19")
  return text

In [23]:
def preprocess(text):
  tok_words = []
  text = text.lower()
  text = text.split(' ')
  for tok_word in text: 
    if "#covid" in tok_word:
      sub_split_pos = tok_word.find('19')
      if sub_split_pos == -1: #coviduk
        if tok_word.strip() == "#covid":
          tok_words.append("covid-19")
        else:
          tok_words.append("covid-19")
          tok_words.append(tok_word[6:])

      elif sub_split_pos+2 == len(tok_word):
        tok_words.append("covid-19")
        #tok_words.append(tok_word.replace("#",""))

      else: #COVID-19Uganda
        fst = tok_word[:sub_split_pos+2]
        tok_words.append("covid-19")
        #tok_words.append(fst.replace("#",""))
        sec = tok_word[sub_split_pos+2:]
        tok_words.append(sec)
    else:
      tok_words.append(tok_word)

  text = ' '.join(tok_words)
  text = text.replace("19","$$")
  text = digit_remove(text)
  text = replace_emoji(text)
  text = text.strip()
  text = text.replace("$$","19")
  text = text.replace("politifact","")
  text = text.replace("forbidden","") 
  text = ' '.join(text.split()) 
  return text

In [24]:
t = "Forbidden PolitiFact 9/11 out🆘 of 🆘 13 people🇯🇵 🇳🇴 🇱🇦 🆘 #COVID--19Updates 🤣 #COVID19 25th Sep 2569 #CovidUK #COVID19Pandemic #covid19ireland #COVID19NZ #covid__19 #covid--19 #covid 19 #COVID-19Nigeria #COVID-19"
t = preprocess(t) 
t

'/ out emergency help required of emergency help required people japan norway laos emergency help required covid-19 updates 🤣 covid-19 th sep covid-19 uk covid-19 pandemic covid-19 ireland covid-19 nz covid-19 covid-19 covid-19 19 covid-19 nigeria covid-19'

In [25]:
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda s : preprocess(s))

In [26]:
test_df.head()

,id,tweet,raw_tweet,clean_tweet
0,1,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22.,our daily update is published. states reported k tests k new cases and deaths. current hospitalizations fell below k for the first time since june .
1,2,Alfalfa is the only cure for COVID-19.,Alfalfa is the only cure for COVID-19.,alfalfa is the only cure for covid-19.
2,3,President Trump Asked What He Would Do If He Were To Catch The Coronavirus https://t.co/3MEWhusRZI #donaldtrump #coronavirus,President Trump Asked What He Would Do If He Were To Catch The Coronavirus #donaldtrump #coronavirus,president trump asked what he would do if he were to catch the coronavirus #donaldtrump #coronavirus
3,4,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process. https://t....,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process.,states reported deaths. we are still seeing a solid national decline. death reporting lags approximately days from symptom onset according to cdc models that consider lags in symptoms time in hospital and the death reporting process.
4,5,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros https://t.co/JvKC0PTett,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros,this is the sixth time a global health emergency has been declared under the international health regulations but it is easily the most severe-@drtedros


In [27]:
tokenizer = CrazyTokenizer(
    keepcaps=True,
    #ignore_stopwords='english',
    #stem = 'stem',
    remove_punct=True, 
    remove_breaks=True,
    decontract=True,
    hashtags='split',
    twitter_handles='realname',
    urls='title',
    normalize=4
)

def concat_tok(text):
  text = tokenizer.tokenize(text)
  text = ' '.join(text)
  text = text.strip()
  return text
test_df['clean_tweet'] = test_df.parallel_apply(lambda x: concat_tok(x['clean_tweet']), axis=1)

   0.00%                                          |        0 /     1070 |      
   0.00%                                          |        0 /     1070 |      M   0.93%                                          |       10 /     1070 |      
   0.00%                                          |        0 /     1070 |      M   1.31%                                          |       14 /     1070 |      
   0.93%                                          |       10 /     1070 |      M   1.31%                                          |       14 /     1070 |      
   5.23% ::                                       |       56 /     1070 |      M   9.35% :::                            

In [29]:
test_df.tail()

,id,tweet,raw_tweet,clean_tweet
2135,2136,#CoronaVirusUpdates: State-wise details of Total Confirmed #COVID19 cases (till 16 September 2020 8 AM) ➡️States with 1-15000 confirmed cases ➡️States with 15001-115000 confirmed cases ➡️States with 115000+ confirmed cases ➡️Total no. of confirmed cas...,#CoronaVirusUpdates: State-wise details of Total Confirmed #COVID19 cases (till 16 September 2020 8 AM) ➡️States with 1-15000 confirmed cases ➡️States with 15001-115000 confirmed cases ➡️States with 115000+ confirmed cases ➡️Total no. of confirmed cas...,coronavirus updates state wise details of total confirmed covid-19 cases till september am ➡ ️states with confirmed cases ➡ ️states with confirmed cases ➡ ️states with + confirmed cases ➡ ️total no of confirmed cases so far
2136,2137,"Tonight 12(midnight) onwards Disaster Management Act has been implemented across the country. According to this update, apart from the Govt department no other citizen is allowed to post any update or share any forward related to Coronavirus, it being...","Tonight 12(midnight) onwards Disaster Management Act has been implemented across the country. According to this update, apart from the Govt department no other citizen is allowed to post any update or share any forward related to Coronavirus, it being...",tonight midnight onwards disaster management act has been implemented across the country according to this update apart from the govt department no other citizen is allowed to post any update or share any forward related to coronavirus it being punish...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 Enugu-46 Oyo-31 Lagos-21 Rivers-20 FCT-15 Kaduna-13 Bauchi-12 Delta-11 Ekiti-11 Akwa Ibom-7 Ebonyi-6 Kwara-5 Ogun-4 Osun-4 Gombe-3 Niger-2 53317 confirmed 40726 discharged 1011 deaths https://t.co/7VLNB37mw1,296 new cases of #COVID19Nigeria; Plateau-85 Enugu-46 Oyo-31 Lagos-21 Rivers-20 FCT-15 Kaduna-13 Bauchi-12 Delta-11 Ekiti-11 Akwa Ibom-7 Ebonyi-6 Kwara-5 Ogun-4 Osun-4 Gombe-3 Niger-2 53317 confirmed 40726 discharged 1011 deaths,new cases of covid-19 nigeria plateau- enugu- oyo- lagos- rivers- fct- kaduna- bauchi- delta- ekiti- akwa ibom- ebonyi- kwara- ogun- osun- gombe- niger- confirmed discharged deaths
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Shop for #COVID19 Resources has a section for communicating with people ages 15-21. Find those t…,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Shop for #COVID19 Resources has a section for communicating with people ages 15-21. Find those t…,rt dyk cdc is one stop shop for covid-19 resources has a section for communicating with people ages find those t
2139,2140,More than half of pregnant women recently admitted to a UK hospital with covid-19 infection were from black or other ethnic minority groups BMJ study finds. https://t.co/rYPBGSNaEL,More than half of pregnant women recently admitted to a UK hospital with covid-19 infection were from black or other ethnic minority groups BMJ study finds.,more than half of pregnant women recently admitted to a uk hospital with covid-19 infection were from black or other ethnic minority groups bmj study finds pregnant women in hospital with covid-19 more likely to be bame


In [30]:
def preprocess_again(text):
  text = re.sub(r"[^A-Za-z0-9|-]", " ", text)  
  text = ' '.join(text.split()) 
  return text
test_df['clean_tweet'] = test_df['clean_tweet'].apply(lambda s : preprocess_again(s))

In [31]:
test_df.to_csv(f'{path}test_data.csv',index=False)

In [32]:
test_df = pd.read_csv(f"{path}test_data.csv")
test_df.head()

,id,tweet,raw_tweet,clean_tweet
0,1,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22.,our daily update is published states reported k tests k new cases and deaths current hospitalizations fell below k for the first time since june
1,2,Alfalfa is the only cure for COVID-19.,Alfalfa is the only cure for COVID-19.,alfalfa is the only cure for covid-19
2,3,President Trump Asked What He Would Do If He Were To Catch The Coronavirus https://t.co/3MEWhusRZI #donaldtrump #coronavirus,President Trump Asked What He Would Do If He Were To Catch The Coronavirus #donaldtrump #coronavirus,president trump asked what he would do if he were to catch the coronavirus donald trump coronavirus
3,4,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process. https://t....,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process.,states reported deaths we are still seeing a solid national decline death reporting lags approximately days from symptom onset according to cdc models that consider lags in symptoms time in hospital and the death reporting process
4,5,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros https://t.co/JvKC0PTett,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros,this is the sixth time a global health emergency has been declared under the international health regulations but it is easily the most severe-


In [33]:
test_poli_list

[91,
 143,
 180,
 239,
 277,
 367,
 457,
 466,
 540,
 557,
 622,
 659,
 711,
 796,
 810,
 811,
 832,
 892,
 962,
 1081,
 1195,
 1198,
 1249,
 1280,
 1323,
 1364,
 1496,
 1535,
 1614,
 1642,
 1759,
 1769,
 1806,
 1808,
 2003,
 2029]

In [34]:
test_df.iloc[91]

id                                                                                                                                                                                                                                                                92
tweet                                                We fact-checked a range of statements from the second night of #DNC2020. Here are highlights dealing with coronavirus case numbers and the #TrumpAdministration’s position on the #ACA. https://t.co/bKhYJteUmf
raw_tweet                                                                   We fact-checked a range of statements from the second night of #DNC2020. Here are highlights dealing with coronavirus case numbers and the #TrumpAdministration’s position on the #ACA. 
clean_tweet    we fact checked a range of statements from the second night of dnc here are highlights dealing with coronavirus case numbers and the trump administration is position on the aca fact checking the second 

In [35]:
test_df.tail()

,id,tweet,raw_tweet,clean_tweet
2135,2136,#CoronaVirusUpdates: State-wise details of Total Confirmed #COVID19 cases (till 16 September 2020 8 AM) ➡️States with 1-15000 confirmed cases ➡️States with 15001-115000 confirmed cases ➡️States with 115000+ confirmed cases ➡️Total no. of confirmed cas...,#CoronaVirusUpdates: State-wise details of Total Confirmed #COVID19 cases (till 16 September 2020 8 AM) ➡️States with 1-15000 confirmed cases ➡️States with 15001-115000 confirmed cases ➡️States with 115000+ confirmed cases ➡️Total no. of confirmed cas...,coronavirus updates state wise details of total confirmed covid-19 cases till september am states with confirmed cases states with confirmed cases states with confirmed cases total no of confirmed cases so far
2136,2137,"Tonight 12(midnight) onwards Disaster Management Act has been implemented across the country. According to this update, apart from the Govt department no other citizen is allowed to post any update or share any forward related to Coronavirus, it being...","Tonight 12(midnight) onwards Disaster Management Act has been implemented across the country. According to this update, apart from the Govt department no other citizen is allowed to post any update or share any forward related to Coronavirus, it being...",tonight midnight onwards disaster management act has been implemented across the country according to this update apart from the govt department no other citizen is allowed to post any update or share any forward related to coronavirus it being punish...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 Enugu-46 Oyo-31 Lagos-21 Rivers-20 FCT-15 Kaduna-13 Bauchi-12 Delta-11 Ekiti-11 Akwa Ibom-7 Ebonyi-6 Kwara-5 Ogun-4 Osun-4 Gombe-3 Niger-2 53317 confirmed 40726 discharged 1011 deaths https://t.co/7VLNB37mw1,296 new cases of #COVID19Nigeria; Plateau-85 Enugu-46 Oyo-31 Lagos-21 Rivers-20 FCT-15 Kaduna-13 Bauchi-12 Delta-11 Ekiti-11 Akwa Ibom-7 Ebonyi-6 Kwara-5 Ogun-4 Osun-4 Gombe-3 Niger-2 53317 confirmed 40726 discharged 1011 deaths,new cases of covid-19 nigeria plateau- enugu- oyo- lagos- rivers- fct- kaduna- bauchi- delta- ekiti- akwa ibom- ebonyi- kwara- ogun- osun- gombe- niger- confirmed discharged deaths
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Shop for #COVID19 Resources has a section for communicating with people ages 15-21. Find those t…,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Shop for #COVID19 Resources has a section for communicating with people ages 15-21. Find those t…,rt dyk cdc is one stop shop for covid-19 resources has a section for communicating with people ages find those t
2139,2140,More than half of pregnant women recently admitted to a UK hospital with covid-19 infection were from black or other ethnic minority groups BMJ study finds. https://t.co/rYPBGSNaEL,More than half of pregnant women recently admitted to a UK hospital with covid-19 infection were from black or other ethnic minority groups BMJ study finds.,more than half of pregnant women recently admitted to a uk hospital with covid-19 infection were from black or other ethnic minority groups bmj study finds pregnant women in hospital with covid-19 more likely to be bame
